In [1]:
# Instalar pacotes essenciais para dados e machine learning
!pip install numpy pandas xlrd pyreadr unidecode matplotlib seaborn scikit-learn xgboost lightgbm statsmodels plotly jupyter notebook
# Instalando pacotes para deep learning
!pip install tensorflow keras torch torchvision 


  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/7c/2f/244643a5ce54a94f0a9a2ab578189c061e4a87c002e037b0829dd77293b6/numpy-2.3.2-cp312-cp312-win_amd64.whl.metadata
     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.9 kB ? eta -:--:--
     ------------------- ------------------ 30.7/60.9 kB 325.1 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/60.9 kB 435.7 kB/s eta 0:00:01
     -------------------------------------- 60.9/60.9 kB 405.6 kB/s eta 0:00:00
  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/80/a5/3a92893e7399a691bad7664d977cb5e7c81cf666c81f89ea76ba2bff483d/pandas-2.3.1-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for xlrd from https://files.pythonhosted.org/packages/1a/62/c8d562e7766786ba6587d09c5a8ba9f718ed3fa8af7f4553e8f91c36f302/xlrd-2.0.2-py2.py3-none-any.whl.metadata
  Obtain


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/5c/98/d145af334fd5807d6ba1ead447bf0c57a36654ea58e726d70c0d09cae913/tensorflow-2.19.0-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for keras from https://files.pythonhosted.org/packages/95/e6/4179c461a5fc43e3736880f64dbdc9b1a5349649f0ae32ded927c0e3a227/keras-3.10.0-py3-none-any.whl.metadata
  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/a1/37/a37495502bc7a23bf34f89584fa5a78e25bae7b8da513bc1b8f97afb7009/torch-2.7.1-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/05/17/e45d5cd3627efdb47587a0634179a3533593436219de3f20c743672d2a79/torchvision-0.22.1-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/8f/aa/ba0014cc4659328dc818a28827be78e6d97312ab0cb98105a770924dc11e/ab


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#Importando bibliotecas
import pandas as pd
import numpy as np
import requests
import os
import xlrd
from unidecode import unidecode
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor


In [3]:

#Carregando dengue bruto
dengue_bruto = pd.read_csv(
    "C:\\Users\\felip\\Downloads\\academia\\pesquisa\\artigos\\previsao_dengue\\dados\\brutos\\1_dengue_casos_19_23\\dengue_casos_19_23.csv"
    , sep=",")

#carregando inmet bruto
inmet_bruto = pd.read_csv(
    "C:\\Users\\felip\\Downloads\\academia\\pesquisa\\artigos\\previsao_dengue\\dados\\brutos\\1_umidade_precipitacao_temperatura_vento\\inmet_2019_2023.csv"
    , sep=",")

#baixando codigos municipais
url = "https://geoftp.ibge.gov.br/organizacao_do_territorio/estrutura_territorial/divisao_territorial/2024/DTB_2024.zip"
response = requests.get(url, stream=True)

#salvando response em um arquivo
with open("C:\\Users\\felip\\Downloads\\academia\\pesquisa\\artigos\\previsao_dengue\\dados\\brutos\\3_coordenadas_municipais\\municipios_codigos_2024.zip", 'wb') as f:
    f.write(response.content)

#descompactando o arquivo zip
import zipfile
with zipfile.ZipFile("C:\\Users\\felip\\Downloads\\academia\\pesquisa\\artigos\\previsao_dengue\\dados\\brutos\\3_coordenadas_municipais\\municipios_codigos_2024.zip", 'r') as zip_ref:
    zip_ref.extractall("C:\\Users\\felip\\Downloads\\academia\\pesquisa\\artigos\\previsao_dengue\\dados\\brutos\\3_coordenadas_municipais")

#carregando RELATORIO_DTB_BRASIL_2024_MUNICIPIOS.xls
#ignornando as 6 primeiras linhas
municipios_codigos_bruto = pd.read_excel(
    "C:\\Users\\felip\\Downloads\\academia\\pesquisa\\artigos\\previsao_dengue\\dados\\brutos\\3_coordenadas_municipais\\RELATORIO_DTB_BRASIL_2024_MUNICIPIOS.xls"
    , usecols=["Código Município Completo", "Nome_Município", "UF", "Nome_UF"], skiprows=6)

In [4]:
#retirar o ultimo caracter de todas as linhas da coluna Código Município Completo
municipios_codigos_bruto['Código Município Completo'] = municipios_codigos_bruto['Código Município Completo'].astype(str).str[:-1]


# Normalize os nomes das cidades (retira acentos e põe tudo em maiúsculo)
inmet_bruto['Nome_Município'] = inmet_bruto['station'].apply(lambda x: unidecode(x.upper()))
municipios_codigos_bruto['Nome_Município'] = municipios_codigos_bruto['Nome_Município'].apply(lambda x: unidecode(x.upper()))
municipios_codigos_bruto.head()

#Criando conjuntos de cidades normalizadas
set1 = set(inmet_bruto['Nome_Município'])
set2 = set(municipios_codigos_bruto['Nome_Município'])
#Verificando interseção e diferenças
# Cidades que existem nos dois DataFrames (serão mantidas no merge)
comuns = set1 & set2

# Cidades que só existem no df1
so_em_inmet_bruto = set1 - set2

# Cidades que só existem no df2
so_em_municipios_codigos_bruto = set2 - set1


#Números
print(f"Total cidades em inmet_bruto: {len(set1)}")
print(f"Total cidades em municipios_codigos_bruto: {len(set2)}")
print(f"Cidades em comum: {len(comuns)}")
print(f"Cidades apenas no inmet_bruto: {len(so_em_inmet_bruto)}")
print(f"Cidades apenas no municipios_codigos_bruto: {len(so_em_municipios_codigos_bruto)}")

#Número de linhas perdidas em um merge
inmet_bruto_filtrado = inmet_bruto[inmet_bruto['Nome_Município'].isin(comuns)]
municipios_codigos_bruto_filtrado = municipios_codigos_bruto[municipios_codigos_bruto['Nome_Município'].isin(comuns)]

print(f"Linhas que permaneceriam no inmet_bruto: {len(inmet_bruto_filtrado)}")
print(f"Linhas que permaneceriam no municipios_codigos_bruto: {len(municipios_codigos_bruto_filtrado)}")

#lista de cidades que não coincidem
print("Exemplos só no inmet_bruto:", list(so_em_inmet_bruto)[:10])
print("Exemplos só no municipios_codigos_bruto:", list(so_em_municipios_codigos_bruto)[:10])

# Agora, faça o merge usando a coluna normalizada de inmet_bruto e municipios_codigos_bruto
inmet_bruto = pd.merge(inmet_bruto, municipios_codigos_bruto, on='Nome_Município', how='inner')

#mudando nome da coluna station para municipio e date para data
inmet_bruto.rename(columns={'date': 'data','Código Município Completo_x':'Nome_Município','Código Município Completo':'ID_municipio'}, inplace=True)

# Convertendo a coluna 'data' para o tipo datetime
inmet_bruto['data'] = pd.to_datetime(inmet_bruto['data'], format='%Y-%m-%d')

#criando nova coluna municipio_data
inmet_bruto['municipio_data'] = inmet_bruto['ID_municipio'] + '_' + inmet_bruto['data'].dt.strftime('%Y-%m-%d')




Total cidades em inmet_bruto: 570
Total cidades em municipios_codigos_bruto: 5290
Cidades em comum: 510
Cidades apenas no inmet_bruto: 60
Cidades apenas no municipios_codigos_bruto: 4780
Linhas que permaneceriam no inmet_bruto: 803009
Linhas que permaneceriam no municipios_codigos_bruto: 563
Exemplos só no inmet_bruto: ['CALCANHAR', 'RIO URUBU', 'CAPAO DO LEAO (PELOTAS)', 'PORTO ALEGRE - JARDIM BOTANICO', 'CAMARATUBA', 'LAGUNA  - FAROL DE SANTA MARTA', 'DUQUE DE CAXIAS - XEREM', 'DELFINO', 'PATY DO ALFERES - AVELAR', 'IBIRITE (ROLA MOCA)']
Exemplos só no municipios_codigos_bruto: ['SAO JORGE DO IVAI', 'VILA NOVA DO PIAUI', 'ZACARIAS', 'CURRAIS', 'ARACATU', 'ITANHANGA', 'MIMOSO DE GOIAS', 'SANTANA', 'FORMOSA DA SERRA NEGRA', 'SANTA FE DE GOIAS']


In [5]:
#Eliminando linha em que as notificações não são individuais
dengue_bruto = dengue_bruto[dengue_bruto["TP_NOT"] == 2]
#Criando coluna casos numero de casos por linha
dengue_bruto['casos'] = 1
# Renomeando colunas
dengue_bruto= dengue_bruto.rename(columns={
    'DT_NOTIFIC': 'data',
    'ID_MUNICIP': 'municipio'
})

# Converter a coluna data para datetime.date
dengue_bruto['data'] = pd.to_datetime(dengue_bruto['data']).dt.date

# Criar um DataFrame de datas completas para cada município
all_dates = pd.date_range(start=dengue_bruto['data'].min(), end=dengue_bruto['data'].max(), freq='D')
all_municipios = dengue_bruto['municipio'].unique()

# Criar um MultiIndex com todas as combinações de município e data
idx = pd.MultiIndex.from_product([all_municipios, all_dates], names=['municipio', 'data'])

# Ordenar os dados para garantir a criação correta de features defasadas
casos = dengue_bruto.sort_values(by=['municipio', 'data']).reset_index(drop=True)

#Somando o numero de casos por município e data
casos =  casos.groupby(['municipio', 'data'])['casos'].sum().reindex(idx, fill_value=0).reset_index()

# Convertendo a coluna 'data' para o tipo datetime
casos['data'] = pd.to_datetime(casos['data'], format='%Y-%m-%d')

# Convertendo a coluna 'municipio' para string
casos['municipio'] = casos['municipio'].astype(str)

#criando nova coluna municipio_data
casos['municipio_data'] = casos['municipio'] + '_' + casos['data'].astype(str)

#remover duas primeiras linhas
casos = casos.iloc[2:].reset_index(drop=True)

In [6]:
#concatenando os dados de inmet_bruto e casos por municipio e data
df = pd.merge(inmet_bruto, casos, on=['municipio_data'], how='inner')
df = df.drop(columns=['municipio', 'data_x','municipio_data','Nome_UF','UF','Nome_Município','station','station_code','uf'])
df.head()

,tair_mean_c,tair_min_c,tair_max_c,dew_tmean_c,dew_tmin_c,dew_tmax_c,rainfall_mm,patm_mb,rh_mean_porc,rh_max_porc,rh_min_porc,ws_2_m_s,sr_mj_m2,ra_mj_m2,longitude_degrees,latitude_degrees,altitude_m,ID_municipio,data_y,casos
0,27.466667,23.3,33.7,22.275000,20.1,24.7,NaN,1004.745833,75.708333,96.0,47.0,NaN,23.7648,36.998709,-40.087288,-3.121067,67.15,230020,2019-01-01,0
1,28.050000,23.4,34.4,21.512500,18.7,24.0,NaN,1005.229167,70.625000,95.0,40.0,0.337500,20.6812,37.015517,-40.087288,-3.121067,67.15,230020,2019-01-02,0
2,27.858333,24.1,33.4,22.808333,21.0,24.4,NaN,1004.545833,76.208333,94.0,49.0,0.333333,21.7821,37.033308,-40.087288,-3.121067,67.15,230020,2019-01-03,0
3,28.429167,24.6,34.6,22.604167,17.9,23.9,NaN,1004.391667,73.791667,94.0,37.0,0.375000,25.9382,37.052055,-40.087288,-3.121067,67.15,230020,2019-01-04,0
4,28.437500,25.0,34.4,23.416667,21.1,24.7,NaN,1004.316667,76.041667,93.0,46.0,0.320833,21.4433,37.071732,-40.087288,-3.121067,67.15,230020,2019-01-05,0


In [8]:

# 3. Criar features temporais
df['dia_semana'] = df['data_y'].dt.dayofweek
df['mes'] = df['data_y'].dt.month
df['ano'] = df['data_y'].dt.year
df['dia_do_ano'] = df['data_y'].dt.dayofyear
df['semana_do_ano'] = df['data_y'].dt.isocalendar().week.astype(int)


In [10]:

# 4. Criar features defasadas (lagged features) para 'casos'
# Isso é crucial para modelos de ML que não são intrinsecamente de séries temporais
# e precisam de informações do passado para prever o futuro.

# Exemplo de lags para 1, 7 e 14 dias (ajuste conforme a necessidade do seu modelo)
for lag in [1, 7, 14]:
    df[f'casos_lag_{lag}'] = df.groupby('ID_municipio')['casos'].shift(lag)

# Preencher valores NaN resultantes dos lags (primeiros dias de cada município)
# Pode-se preencher com 0 ou com a média/mediana, dependendo do contexto.
# Para casos de dengue, 0 pode ser uma suposição razoável para dias sem casos anteriores.
df.fillna(0, inplace=True)


In [11]:
#aplicando log(x+1) para a variavel casos
df['casos_log'] = np.log(df['casos'] + 1)

In [12]:
#tornando ID_municipio uma varivel categorica dummy
dummies = pd.get_dummies(df['ID_municipio'], prefix='mun')
df = pd.concat([df, dummies], axis=1)

In [ ]:
# Random Forest
# 2. Definir variáveis
X = df.drop(columns=['casos', 'casos_log','data_y','ID_municipio'])  # Features
y = df['casos_log']  # Alvo log-transformado

# 3. Dividir treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Treinar Random Forest
rf = RandomForestRegressor(n_estimators=50, random_state=42)
rf.fit(X_train, y_train)

# 5. Previsões na escala log
y_pred_log = rf.predict(X_test)

# 6. Destransformar para a escala original de 'casos'
y_pred = np.exp(y_pred_log) - 1
y_true = np.exp(y_test) - 1

# 7. Avaliar o modelo na escala original
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
r2 = r2_score(y_true, y_pred)

print(f"RMSE (escala original): {rmse:.4f}")
print(f"R² (escala original): {r2:.4f}")
